In [88]:
import pandas as pd

MILANO = pd.read_csv("../Dataset/Comune-di-Milano-Pubblici-esercizi(in)-2.csv",sep=';',encoding='utf-8')
MILANO.head()

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
0,NaN,NaN,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0
1,NaN,NaN,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0
2,NaN,NaN,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0
3,NaN,NaN,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0
4,NaN,NaN,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0


In [89]:
MILANO_CLEAN = MILANO[["Ubicazione", "ZD"]].copy()
MILANO_CLEAN.head()

,Ubicazione,ZD
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6


In [90]:
regex = r"z\.d\.\s*(\d+)"
MILANO_CLEAN["Ubi ZD"] = MILANO_CLEAN["Ubicazione"].str.extract(regex)
MILANO_CLEAN.head(100)

,Ubicazione,ZD,Ubi ZD
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6
...,...,...,...
95,PLE MACIACHINI CARLO N. 1 (z.d. 9),9,9
96,PLE PERRUCCHETTI GIUSEPPE N. 4 (z.d. 7),7,7
97,PLE PRINCIPESSA CLOTILDE N. 10 (z.d. 9),9,9
98,PLE STAZIONE DI PORTA GENOVA N. 2 (z.d. 6),6,6


In [91]:
nulls = MILANO_CLEAN["Ubi ZD"].isnull().sum()
MILANO_CLEAN["Ubi ZD"] = pd.to_numeric(MILANO_CLEAN["Ubi ZD"], errors="coerce")
nulls

0

In [92]:
MILANO_CLEAN["ZD_match"] = MILANO_CLEAN["ZD"] == MILANO_CLEAN["Ubi ZD"]
MILANO_CLEAN.head()

,Ubicazione,ZD,Ubi ZD,ZD_match
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6,True
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6,True
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6,True
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6,True
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6,True


In [93]:
different = (MILANO_CLEAN["ZD_match"] == "False").sum()
different

0

In [94]:
MILANO_CLEAN["ZD_match"].mean()

0.9955098493626883

In [95]:
idx_mismatch_zd = MILANO_CLEAN.index[MILANO_CLEAN["ZD_match"] == False]
len(idx_mismatch_zd)

31

Which representation of ZD is more reliable, and can Codice Via be used as external evidence to support that choice?
Given the large cardinality difference between Codice Via and ZD, we tested whether Codice Via could act as a determinant for ZD. The analysis showed that streets overwhelmingly map to a single ZD, enabling us to use Codice Via as an independent reference to evaluate conflicting ZD representations.

In [96]:
MILANO_CLEAN["Codice via"] = MILANO["Codice via"].copy()
MILANO_CLEAN.head()

,Ubicazione,ZD,Ubi ZD,ZD_match,Codice via
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6,True,5144
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6,True,5144
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6,True,5144
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6,True,5144
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6,True,5161


In [97]:
MILANO_CLEAN.groupby(MILANO_CLEAN["Codice via"])["ZD"].nunique()
duplicate_zd = (MILANO_CLEAN.groupby(MILANO_CLEAN["Codice via"])["ZD"].nunique() > 1).sum() 
duplicate_zd

67

If there is a Codice Via with > 1 ZD, it is suspicious. I am going to make the same query using the zd from the Ubicazione and compare the results

In [98]:
MILANO_CLEAN.groupby(MILANO_CLEAN["Codice via"])["Ubi ZD"].nunique()
duplicate_ubi_zd = (MILANO_CLEAN.groupby(MILANO_CLEAN["Codice via"])["Ubi ZD"].nunique() > 1).sum() 
duplicate_ubi_zd

70

Both ZD and the ZD extracted from Ubicazione exhibit high internal consistency when evaluated against Codice via. However, the structured ZD attribute shows slightly fewer intra-street inconsistencies (67 vs 70). Combined with its structured nature and lower susceptibility to text-level errors, ZD is therefore considered the more reliable source.

In [99]:
# Now I am going to find all the rows in the table in which Ubicazione is not aligned with the other columns. To do that I will extract the different parts of the Ubicazione field and compare each part with the respective 
# column. If the Tipo and Descrizione are different then the whole value has to be changed. If it is only the Civico or the ZD it stays the same and just change it

MILANO_CLEAN["Ubi tipo"] = MILANO_CLEAN["Ubicazione"].str.extract(r"^(.{3})")
MILANO_CLEAN["Ubi tipo"] = MILANO_CLEAN["Ubi tipo"].str.strip().str.upper()
MILANO_CLEAN["Ubi tipo"].head(30)

0     ALZ
1     ALZ
2     ALZ
3     ALZ
4     ALZ
5     ALZ
6     BST
7     BST
8     BST
9     BST
10    COD
11    COD
12    CSO
13    CSO
14    CSO
15    CSO
16    CSO
17    CSO
18    CSO
19    CSO
20    CSO
21    CSO
22    CSO
23    CSO
24    CSO
25    CSO
26    CSO
27    CSO
28    CSO
29    CSO
Name: Ubi tipo, dtype: object

In [100]:
# Now extracting the Civico
regex = r"\bN\.?\s*([0-9])"
MILANO_CLEAN["Ubi civico"] = MILANO_CLEAN["Ubicazione"].str.extract(regex)
#MILANO_CLEAN["Ubi civico"].head(100)
idx_mismatch = MILANO_CLEAN.index[MILANO_CLEAN["Ubi civico"].isnull()]
idx_mismatch

Index([   9,   10,   11,   45,   46,   51,  118,  140,  141,  159,
       ...
       6581, 6583, 6585, 6673, 6706, 6781, 6799, 6829, 6861, 6868],
      dtype='int64', length=265)

In [101]:
# I realized that there could be another marker for the civico that is "num."

regex = r"\b(?:N\.?|num\.)\s*0*([0-9]+)"
MILANO_CLEAN["Ubi civico"] = MILANO_CLEAN["Ubicazione"].str.extract(regex)
#MILANO_CLEAN["Ubi civico"].head(100)
idx_mismatch = MILANO_CLEAN.index[MILANO_CLEAN["Ubi civico"].isnull()]
idx_mismatch

Index([10, 11, 2899], dtype='int64')

With this addition, the number of NaN drops to 3, most likely because of the error in the Ubicazione field. Just to be sure I will check manually. It confirmed my suspicions 

In [102]:
MILANO_CLEAN["Ubicazione"].loc[MILANO_CLEAN.index[2899]]
#(data['id'].loc[data.index[0]]) 

"codvia 7105 (c/o 'torre branca'); accesso: accesso esterno; (z.d. 1)"

In [103]:
# Now it is the Descrizione turn 
regex = r"^.{3}\s+(.*?)\s+\b(?:N\.?|num\.)\b"
MILANO_CLEAN["Ubi descrizione via"] = MILANO_CLEAN["Ubicazione"].str.extract(regex)
idx_null = MILANO_CLEAN.index[MILANO_CLEAN["Ubi descrizione via"].isnull()]
idx_null

Index([10, 11, 2899, 5929], dtype='int64')

In [104]:
MILANO_CLEAN["Ubi tipo"].loc[MILANO_CLEAN.index[5929]]

'LAR'

In [105]:
MILANO_CLEAN.head(150)

,Ubicazione,ZD,Ubi ZD,ZD_match,Codice via,Ubi tipo,Ubi civico,Ubi descrizione via
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6,True,5144,ALZ,12,NAVIGLIO GRANDE
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6,True,5144,ALZ,44,NAVIGLIO GRANDE
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6,True,5144,ALZ,48,NAVIGLIO GRANDE
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6,True,5144,ALZ,8,NAVIGLIO GRANDE
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6,True,5161,ALZ,24,NAVIGLIO PAVESE
...,...,...,...,...,...,...,...,...
145,PZA MONTE FALTERONA N. 1 (z.d. 7),7,7,True,6558,PZA,1,MONTE FALTERONA
146,PZA MONTE FALTERONA N. 6 (z.d. 7),7,7,True,6558,PZA,6,MONTE FALTERONA
147,PZA NAPOLI N. 19 (z.d. 6),6,6,True,5131,PZA,19,NAPOLI
148,PZA OBERDAN GUGLIELMO N. 2 (z.d. 1),1,1,True,2108,PZA,2,OBERDAN GUGLIELMO


In [106]:
# Now I will start with the checks: starting from the Tipo
MILANO_CLEAN["Tipo_match"] = MILANO_CLEAN["Ubi tipo"] == MILANO["Tipo via"]
idx_mismatch_tipo = MILANO_CLEAN.index[MILANO_CLEAN["Tipo_match"] == False]
idx_mismatch_tipo

Index([  10,   11,   46,  140,  141,  305,  467,  579,  655, 1078, 1177, 1204,
       1282, 1287, 1288, 1348, 1503, 1701, 1716, 1724, 1856, 1927, 2479, 2558,
       2877, 2899, 2952, 3077, 3319, 3348, 3379, 3695, 4184, 4620, 4667, 4784,
       5090, 5254, 5356, 5488, 5571, 5719, 5767, 5792, 5797, 5871, 5902, 5929,
       6021, 6033, 6037, 6174, 6205, 6290, 6578, 6868],
      dtype='int64')

In [107]:
MILANO_CLEAN["Civico_match"] = MILANO_CLEAN["Ubi civico"] == MILANO["Civico"]
idx_mismatch_civico = MILANO_CLEAN.index[MILANO_CLEAN["Civico_match"] == False]
idx_mismatch_civico

Index([   9,   10,   11,   45,   46,   51,  118,  140,  141,  159,
       ...
       6581, 6583, 6585, 6673, 6706, 6781, 6799, 6829, 6861, 6868],
      dtype='int64', length=268)

In [108]:
MILANO_CLEAN["Descrizione_match"] = MILANO_CLEAN["Ubi descrizione via"] == MILANO["Descrizione via"]
idx_mismatch_desc = MILANO_CLEAN.index[MILANO_CLEAN["Descrizione_match"] == False]
idx_mismatch_desc

Index([  10,   11,   45,   46,   51,  140,  141,  305,  362,  373,
       ...
       6037, 6111, 6174, 6205, 6254, 6287, 6290, 6399, 6578, 6868],
      dtype='int64', length=108)

In [109]:
MILANO_CLEAN

,Ubicazione,ZD,Ubi ZD,ZD_match,Codice via,Ubi tipo,Ubi civico,Ubi descrizione via,Tipo_match,Civico_match,Descrizione_match
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6,True,5144,ALZ,12,NAVIGLIO GRANDE,True,True,True
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6,True,5144,ALZ,44,NAVIGLIO GRANDE,True,True,True
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6,True,5144,ALZ,48,NAVIGLIO GRANDE,True,True,True
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6,True,5144,ALZ,8,NAVIGLIO GRANDE,True,True,True
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6,True,5161,ALZ,24,NAVIGLIO PAVESE,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
6899,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,2,3,False,2230,VLE,12,DORIA ANDREA,True,True,True
6900,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,9,9,True,1134,VIA,5,GARIGLIANO,True,True,True
6901,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,4,4,True,3152,VIA,4,SOTTOCORNO PASQUALE,True,True,True
6902,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,7,7,True,6299,VIA,23,CASTROVILLARI,True,True,True


In [110]:
MILANO_CLEAN["ubi_new"] = MILANO_CLEAN["Ubicazione"]

ubi_correct = (
    MILANO["Tipo via"].astype(str).str.strip() + " " +
    MILANO["Descrizione via"].astype(str).str.strip() + " N. " +
    MILANO["Civico"].astype(str).str.strip() + " " +
    "(z.d. " + MILANO["ZD"].astype("Int64").astype(str) + ")"
).str.replace(r"\s+", " ", regex=True).str.strip()

mask_desc_mismatch = ~MILANO_CLEAN["Descrizione_match"]
MILANO_CLEAN.loc[mask_desc_mismatch, "ubi_new"] = ubi_correct.loc[mask_desc_mismatch]

MILANO_CLEAN

,Ubicazione,ZD,Ubi ZD,ZD_match,Codice via,Ubi tipo,Ubi civico,Ubi descrizione via,Tipo_match,Civico_match,Descrizione_match,ubi_new
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),6,6,True,5144,ALZ,12,NAVIGLIO GRANDE,True,True,True,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6)
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),6,6,True,5144,ALZ,44,NAVIGLIO GRANDE,True,True,True,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6)
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),6,6,True,5144,ALZ,48,NAVIGLIO GRANDE,True,True,True,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6)
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),6,6,True,5144,ALZ,8,NAVIGLIO GRANDE,True,True,True,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6)
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),6,6,True,5161,ALZ,24,NAVIGLIO PAVESE,True,True,True,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6)
...,...,...,...,...,...,...,...,...,...,...,...,...
6899,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,2,3,False,2230,VLE,12,DORIA ANDREA,True,True,True,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...
6900,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,9,9,True,1134,VIA,5,GARIGLIANO,True,True,True,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...
6901,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,4,4,True,3152,VIA,4,SOTTOCORNO PASQUALE,True,True,True,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...
6902,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,7,7,True,6299,VIA,23,CASTROVILLARI,True,True,True,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...


In [111]:
# Now that I have changed the ubicazione, I will test the other fields as well to see if much has changed. Zero for Civico
regex = r"\b(?:N\.?|num\.)\s*0*([0-9]+)"
MILANO_CLEAN["Ubi civico"] = MILANO_CLEAN["ubi_new"].str.extract(regex)
#MILANO_CLEAN["Ubi civico"].head(100)
idx_mismatch = MILANO_CLEAN.index[MILANO_CLEAN["Ubi civico"].isnull()]
idx_mismatch

Index([], dtype='int64')

In [112]:
MILANO_CLEAN["Tipo_match"] = MILANO_CLEAN["Ubi tipo"] == MILANO["Tipo via"]
idx_mismatch_tipo = MILANO_CLEAN.index[MILANO_CLEAN["Tipo_match"] == False]
idx_mismatch_tipo

Index([  10,   11,   46,  140,  141,  305,  467,  579,  655, 1078, 1177, 1204,
       1282, 1287, 1288, 1348, 1503, 1701, 1716, 1724, 1856, 1927, 2479, 2558,
       2877, 2899, 2952, 3077, 3319, 3348, 3379, 3695, 4184, 4620, 4667, 4784,
       5090, 5254, 5356, 5488, 5571, 5719, 5767, 5792, 5797, 5871, 5902, 5929,
       6021, 6033, 6037, 6174, 6205, 6290, 6578, 6868],
      dtype='int64')